In [10]:
import numpy as np
import pandas as pd


In [3]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(10682, 16)

In [4]:
len(df[df.tiling_compatible == True]) 

524

In [5]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   5009
1         group_structure   1212
2  tile_generation_failed   1107
3   failed_to_extract_url    962
4        no_xy_dimensions    886
5          cant_open_file    866
6                    None    546
7  cant_extract_variables     94


In [6]:
df[df.incompatible_reason == 'tile_generation_failed'].error_message.unique()


array(['list index out of range',
       'Expecting value: line 1 column 1 (char 0)',
       "b'CDF\\x01\\x00\\x00\\x00\\x00' is not the signature of a valid netCDF4 file",
       'titiler.xarray can only work with 2D or 3D dataset',
       'Unable to allocate 15.6 GiB for an array with shape (12, 13165, 26477) and data type float32',
       'No assets found for tile 0-0-0',
       "b'CDF\\x01\\x00\\x00\\x01m' is not the signature of a valid netCDF4 file",
       "b'CDF\\x01\\x00\\x00\\x00\\x0c' is not the signature of a valid netCDF4 file",
       "b'CDF\\x01\\x00\\x00\\x00\\xf0' is not the signature of a valid netCDF4 file",
       "No group found in store <fsspec.mapping.FSMap object at 0x7f554a6ee5d0> at path ''",
       "b'CDF\\x01\\x00\\x00M\\x19' is not the signature of a valid netCDF4 file",
       'Assets HAVE TO have the same number of bands',
       "b'CDF\\x01\\x00\\x00\\x13\\x08' is not the signature of a valid netCDF4 file",
       '"No variable named \'altitude\'. Variab

In [16]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (len(x) > 0 and x[0] != None)
)]
subset_df

,collection_concept_id,concept_id,data_center,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables,num_granules,processing_level,groups
0,C1000000060-LARC_ASDC,G1533371667-LARC_ASDC,ASDC,https://asdc.larc.nasa.gov/data/CERES/GEO/Edit...,xarray,NETCDF-4,nc,False,tile_generation_failed,list index out of range,https://staging.openveda.cloud/api/titiler-cmr...,vis_063_calib,"[vis_063_calib, temp_nir_calib, temp_ir_calib,...",48317.0,2,None
106,C1000000922-LARC_ASDC,G1000713545-LARC_ASDC,ASDC,https://asdc.larc.nasa.gov/data/FIRE/FIRE_ACE_...,xarray,NETCDF-4,cdf,False,tile_generation_failed,list index out of range,https://staging.openveda.cloud/api/titiler-cmr...,Wavelength,"[Wavelength, Time_Data, Irradiance]",21.0,2,None
309,C1231627223-GES_DISC,G1289992992-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/TRMM_RT/TR...,xarray,netCDF,nc4,False,tile_generation_failed,Expecting value: line 1 column 1 (char 0),https://staging.openveda.cloud/api/titiler-cmr...,precipitation,"[precipitation, precipitation_error, source, u...",57961.0,3,None
310,C1233767545-GES_DISC,G1415044909-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/GLDAS/GLDA...,xarray,netCDF,nc4,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Swnet_tavg,"[time_bnds, Swnet_tavg, Lwnet_tavg, Qle_tavg, ...",195775.0,4,None
311,C1233767548-GES_DISC,G1414475414-GES_DISC,GES_DISC,s3://gesdisc-cumulus-prod-protected/GLDAS/GLDA...,xarray,netCDF,nc4,True,None,None,https://staging.openveda.cloud/api/titiler-cmr...,Swnet_tavg,"[time_bnds, Swnet_tavg, Lwnet_tavg, Qle_tavg, ...",195775.0,4,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10618,C3826284331-NSIDC_CPRD,G3838381861-NSIDC_CPRD,NSIDC,s3://nsidc-cumulus-prod-protected/ATLAS/ATL21/...,xarray,None,h5,False,no_xy_dimensions,Couldn't find X/Y dimensions in (),https://staging.openveda.cloud/api/titiler-cmr...,crs,"[crs, grid_lat, grid_lon, land_mask_map]",162.0,3,None
10620,C3834287411-ORNL_CLOUD,G3835158118-ORNL_CLOUD,ORNLDAAC,s3://ornl-cumulus-prod-protected/shift/SHIFT_A...,xarray,netCDF-4,nc,False,no_xy_dimensions,Couldn't find X/Y dimensions in (),https://staging.openveda.cloud/api/titiler-cmr...,transverse_mercator,[transverse_mercator],1593.0,2,None
10621,C3834287416-ORNL_CLOUD,G3845354168-ORNL_CLOUD,ORNLDAAC,s3://ornl-cumulus-prod-protected/shift/SHIFT_A...,xarray,netCDF-4,nc,False,no_xy_dimensions,"Couldn't find X/Y dimensions in ('lines', 'sam...",https://staging.openveda.cloud/api/titiler-cmr...,lat,"[lat, lon, elev, transverse_mercator]",1598.0,1A,None
10623,C3838944284-GHRC_DAAC,G3842115505-GHRC_DAAC,GHRCDAAC,s3://ghrcw-protected/isslis_v3_fin__3/ISS_LIS_...,xarray,NETCDF-4,nc,False,tile_generation_failed,b'\x0e\x03\x13\x01\x01\x00\x00\x0e' is not the...,https://staging.openveda.cloud/api/titiler-cmr...,raster_image,"[raster_image, raster_image_color_table, orbit...",72828.0,2,None


In [25]:
result = subset_df[(subset_df.incompatible_reason == 'tile_generation_failed') & 
                   (subset_df['error_message'].str.contains('No variable named', na=False))]

In [26]:
result

,collection_concept_id,concept_id,data_center,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables,num_granules,processing_level,groups
2711,C1979602587-GHRC_DAAC,G1979602676-GHRC_DAAC,GHRCDAAC,s3://ghrcw-protected/gpmhiwrapiphx__1/IPHEX_HI...,xarray,NETCDF-4,nc,False,tile_generation_failed,"""No variable named 'altitude'. Variables on th...",https://staging.openveda.cloud/api/titiler-cmr...,altitude,"[altitude, dopcorr, evel, gatesp, gspeed, head...",210.0,1B,None
3156,C1995568158-GHRC_DAAC,G2768006300-GHRC_DAAC,GHRCDAAC,s3://ghrcw-protected/goesimpacts__1/IMPACTS_go...,xarray,NETCDF-4,nc,False,tile_generation_failed,"""No variable named 'Rad'. Variables on the dat...",https://staging.openveda.cloud/api/titiler-cmr...,Rad,"[Rad, DQF, time_bounds, goes_imager_projection...",848057.0,3,None
3194,C1995869065-GHRC_DAAC,G1997644304-GHRC_DAAC,GHRCDAAC,s3://ghrcw-protected/sbuceilimpacts__1/IMPACTS...,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'latitude'. Did you mean on...",https://staging.openveda.cloud/api/titiler-cmr...,latitude,"[latitude, longitude, azimuth, zenith, altitud...",15836.0,2,None
4121,C2216863856-ORNL_CLOUD,G2217349171-ORNL_CLOUD,ORNLDAAC,s3://ornl-cumulus-prod-protected/global_soil/H...,xarray,None,nc4,False,tile_generation_failed,"""No variable named 'S_PH_H2O'. Did you mean on...",https://staging.openveda.cloud/api/titiler-cmr...,S_PH_H2O,[S_PH_H2O],28.0,4,None
4314,C2254805714-POCLOUD,G2777006404-POCLOUD,PODAAC,s3://podaac-ops-cumulus-protected/SAILDRONE_AR...,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'roll'. Variables on the da...",https://staging.openveda.cloud/api/titiler-cmr...,roll,"[roll, pitch, heading, vel_east, vel_north, ve...",4.0,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10461,C3620140295-OB_CLOUD,G3748132880-OB_CLOUD,OBDAAC,s3://ob-cumulus-prod-public/PACE_OCI.20250419....,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'aph_unc_442'. Did you mean...",https://staging.openveda.cloud/api/titiler-cmr...,aph_unc_442,"[aph_unc_442, palette]",14580.0,3,None
10585,C3779578929-OB_CLOUD,G3812672335-OB_CLOUD,OBDAAC,s3://ob-cumulus-prod-public/JPSS2_VIIRS.202510...,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'adg_s'. Variables on the d...",https://staging.openveda.cloud/api/titiler-cmr...,adg_s,"[adg_s, palette]",2772.0,3,None
10586,C3779578995-OB_CLOUD,G3801383694-OB_CLOUD,OBDAAC,s3://ob-cumulus-prod-public/JPSS2_VIIRS.202408...,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'a_411'. Variables on the d...",https://staging.openveda.cloud/api/titiler-cmr...,a_411,"[a_411, palette]",41868.0,3,None
10595,C3779579730-OB_CLOUD,G3799232446-OB_CLOUD,OBDAAC,s3://ob-cumulus-prod-public/JPSS2_VIIRS.202509...,xarray,netCDF-4,nc,False,tile_generation_failed,"""No variable named 'angstrom'. Variables on th...",https://staging.openveda.cloud/api/titiler-cmr...,angstrom,"[angstrom, palette]",1078.0,3,None


In [ ]:
all_unique_vars = set().union(*result.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs', 'missing', 'palette', 'scaling', 'base', 'profile', 'error_ext', 'error_vel']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

539


In [29]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
